## 使用 TensorBoard
### 即時查看參數變化情況
1. 在程式目錄下建立一個floder存放Tensorboard的紀錄文件
2. 在程式碼中宣告一個writer
    ```python
    # 參數為記錄文件所保存的目錄
    summary_writer = tf.summary.create_file_writer('./tensorboard')     
    ```
3. 接下來當需要記錄訓練過程中的參數時，可利用 **with** 來指定要使用的writer，並對需要記錄的參數(一般是scalar)運行，即可將訓練過程中使參數在每個step時的值記錄下來。這裡的step可根據自己的需要自行訂製，一般可設置為當前訓練過程中的batch index:
    ```python
    tf.summary.scalar(name, tensor, step=batch_index)
    ```
    * 範例:
    ```python
    summary_writer = tf.summary.create_file_writer('./tensorboard')
    # 開始模型訓練
    for batch_index in range(num_batches):
        # ...（訓練程式碼，loss被存在loss中）
        with summary_writer.as_default(): # 希望使用的writer
            tf.summary.scalar("loss", loss, step=batch_index)
            tf.summary.scalar("MyScalar", my_scalar, step=batch_index)
    ```
    * 每運行一次tf.summary.scalar()，writer就會向記錄文件中寫入一條記錄。
    * 除了scalar以外，TensorBoard 還可以對其他類型的資料（如圖片，音頻等）進行視覺化
4. 接著利用terminal輸入 **tensorboard --logdir=./tensorboard**
5. 透過瀏覽器訪問tensorboard頁面

#### 注意事項
1. 如果需要重新訓練，需要刪除掉記錄資料夾內的資訊並重啟TensorBoard(或者建立一個新的記錄資料夾並開啟TensorBoard)
2. 記錄資料夾目錄需保持全英文

### 查看 Graph 和 Profile
* 在訓練時，可使用 **tf.summary.trace_on** 去開啟Trace，此時TensorFlow會將訓練時的大量資訊(如計算圖的結構、每個運作所耗費的時間等)記錄下來
* 在訓練完後，使用 **tf.summary.trace_export** 將紀錄及果輸出到文件上
```python
# 開啟Trace，可以記錄圖結構和profile資訊
tf.summary.trace_on(graph=True, profiler=True)  
# 進行訓練
with summary_writer.as_default():
    # 保存Trace資訊到文件
    tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=log_dir)    
```
* 之後即可在tensorboard查看視覺化後的訓練資訊

[參考網站https://tf.wiki/zh_hant/basic/tools.html](https://tf.wiki/zh_hant/basic/tools.html)

## 利用MLP實作

In [1]:
import tensorflow as tf
import numpy as np
from mlp import MLP #import先前製作好的模型架構
from mlp import MNISTLoader

In [2]:
# 定義 hyperparameter
num_batches = 1000
batch_size = 50
learning_rate = 0.001
# 紀錄儲存的目錄
log_dir = 'tensorboard'
# 模型物件宣告、資料集載入、優化器宣告
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [3]:
# .....................................................
# 宣告 writer
summary_writer = tf.summary.create_file_writer(log_dir)     
tf.summary.trace_on(graph=True, profiler=True)  # 開啟Trace記錄圖結構和profile資訊
# .....................................................

Instructions for updating:
use `tf.profiler.experimental.start` instead.


In [4]:
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        if (batch_index+1)%100 == 0:
            print("batch %d: loss %f" % (batch_index, loss.numpy()))
        # .....................................................
        # 指定 writer
        with summary_writer.as_default():  
            # 將目前的loss寫入
            tf.summary.scalar("loss", loss, step=batch_index)  
        # .....................................................
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 99: loss 0.651963
batch 199: loss 0.193828
batch 299: loss 0.303658
batch 399: loss 0.331358
batch 499: loss 0.126272
batch 599: loss 0.096747
batch 699: loss 0.436470
batch 799: loss 0.085009
batch 899: loss 0.213919
batch 999: loss 0.354831


In [5]:
# .....................................................
# 儲存Trace資料到文件
with summary_writer.as_default():
    tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=log_dir)    
# .....................................................

Instructions for updating:
use `tf.profiler.experimental.stop` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.
Instructions for updating:
`tf.python.eager.profiler` has deprecated, use `tf.profiler` instead.


### 開啟 tensorboard
* 輸入指令
![image.png](img/tb01.png)
* 查看loss
![image-2.png](img/tb02.png)
* 查看profile
![image-3.png](img/tb03.png)

* 更細節可[參考https://learn.microsoft.com/zh-tw/cognitive-toolkit/using-tensorboard-for-visualization](https://learn.microsoft.com/zh-tw/cognitive-toolkit/using-tensorboard-for-visualization)